In [3]:
from typing import List
from pydantic import BaseModel
from openai import OpenAI
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
TRANSCRIPT = """\
I’d like to about a person I really admire, and that is my elder.

She’s about five years older than me, and I’ve know her my whole life, obviously, but we became much closer when I was in high school. That was the time when I was quite confused about my future, and she was already working, so I used to talk to her a lot.

One I really admire in her is her sense of responsibility. No matter how tired she is, she always finishes what she starts. She’s also very calm, which is the complete opposite of me. When there’s a problem, she doesn’t panic; instead, she tries to find a practical solution. I think that’s a very useful skill in real life.

Another thing I admire is that she’s very independent. She moved to another city for her and managed everything on her own, from rent to finances, without complaining much. Watching her made me realize that being independent isn’t easy, but it’s definitely worth it.

I admire her mainly because she motivates me without forcing me. She never lectures me, but her actions push me to work harder and be more disciplined. Honestly, whenever I feel lazy, I just think about how she handles her life, and that gives me a reality check.
"""

In [6]:
prompt_text = TRANSCRIPT

In [7]:
system_prompt = """
You are an expert grammar analysis engine specialized in spoken English assessment.

Your task:
- Analyze the transcript as natural spoken English.
- Detect grammatical errors that would be penalized in formal spoken English evaluation (e.g. IELTS Speaking).
- Identify errors precisely and conservatively.

Error categories:
- tense
- agreement
- articles
- prepositions
- sentence_structure
- other

Rules:
- Ignore pronunciation issues.
- Ignore vocabulary choice unless grammatically incorrect.
- Do NOT correct the text.
- Do NOT suggest fixes.
- Do NOT explain grammar rules.
- Do NOT infer intended meaning beyond what is grammatically present.
- Treat omissions (missing required words or structures) as grammatical errors.
- Count each distinct grammatical error once.

For each detected error:
- Provide the minimal text span that contains the error.
- Include the sentence index (0-based).
- Include character start and end positions relative to the full transcript.
- Do NOT include corrections or explanations.

Analysis steps:
1. Determine total sentence count (spoken-style sentences included).
2. Identify all grammatical errors.
3. Classify each error by type.
4. Ensure counts match the listed errors.

"""


In [8]:
class GrammarError(BaseModel):
    type: str
    context: str
    sentence_index: int
    char_start: int
    char_end: int


class ErrorsByType(BaseModel):
    tense: int
    agreement: int
    articles: int
    prepositions: int
    sentence_structure: int
    other: int


class GrammarAnalysis(BaseModel):
    sentence_count: int
    error_count: int
    errors_by_type: ErrorsByType
    errors: List[GrammarError]

In [9]:
client = OpenAI()

response = client.responses.parse(
    model="gpt-4o-mini",  
    input=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt_text},
    ],
    text_format=GrammarAnalysis,  # 🔒 schema enforced here
)

grammar_data: GrammarAnalysis = response.output_parsed
for err in grammar_data.errors:
    print(err.type, err.context)

sentence_structure I’d like to about a person I really admire, and that is my elder.
